# INDIVIDUAL FOOTPRINTS

In [1]:
import numpy as np
import datetime
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt

## Preparacion de datos

### Cargando datos

In [2]:

def leer_data():
    outfile='./data2/data.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data_original.head(3)

,client_id,date,año,mes,dia,hora,merchant_departement,merchant_province,merchant_district,mcc,mccg,client_age,quantity,amount_sol,dia_semana,turno
0,+++g8j9k+5A=,2016-07-07 01:08:05,2016,7,7,1,LIMA,LIMA,San Juan De Luriganc,5655,8,32.0,1,45.0,Thurs,1-Madrugada
1,++/oQ9Lb9dI=,2017-05-29 21:05:40,2017,5,29,21,LIMA,LIMA,San Juan De Luriganc,4812,25,49.0,1,104.0,Mon,4-Noche
2,++14g8obpj0=,2016-09-08 21:15:55,2016,9,8,21,LIMA,LIMA,San Juan De Luriganc,5912,15,NaN,1,9.0,Thurs,4-Noche


### Preparacion de datos

In [3]:
data = data_original[['client_id','date','año','mes','dia','hora','quantity','amount_sol']]
data.head(3)   

,client_id,date,año,mes,dia,hora,quantity,amount_sol
0,+++g8j9k+5A=,2016-07-07 01:08:05,2016,7,7,1,1,45.0
1,++/oQ9Lb9dI=,2017-05-29 21:05:40,2017,5,29,21,1,104.0
2,++14g8obpj0=,2016-09-08 21:15:55,2016,9,8,21,1,9.0


## Unidad de TXs temporales (U)

In [4]:
file='U'
raw_data='./data2/%s.json' %(file)

### Funciones

In [5]:
# definimos los 4 time_windows que usaremos

def time_window(hora):
    tw = 9999
    if hora >=0:
        tw = 0      # Madrugada
    if hora >=6:
        tw = 1      # Mañana
    if hora >=12:
        tw = 2      # Tarde
    if hora >=18:
        tw = 3      # Noche
    return tw

In [6]:
# Definimos los U 

def procesar_u(user):    
    uid=list(user['client_id'])[0]
    
    # Lista los años en que tiene txs el usuario
    years = set(list(user['año']))
    anni = {year:{} for year in list(years)}
    
    # para cada fila (para cada fecha)
    for dat in  range(0,len(user)):
        año = user.iloc[dat]['año']
        week=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).isocalendar()[1]
        weekday=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).weekday()
        turn = time_window(data.iloc[dat]['hora'])
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week] = {}
        # Si el turno no existe en la semana y año
        if not (turn in anni[año][week]):
            anni[año][week][turn]=np.array([0]*7)  #CUATRO TURNOS
            
        anni[año][week][turn][weekday]+=user.iloc[dat]['quantity'] # suma cantidades "importancia por compras"
        #anni[año][week][turn][weekday]+=user.iloc[dat]['amount_sol'] # suma montos "importancia por gastos"
            
    return uid,anni

### Procesando U de cada cliente

In [ ]:
# Extraemos la lista de clientes sin repetir
clientes =  data.groupby('client_id').client_id.count().index

profiles={}
# Para cada cliente
i = 0
for cliente in clientes:
    cliente_i= data[data['client_id'] == cliente]
    ## ejecutamos para cada usuario
    results=procesar_u(cliente_i)
    profiles[results[0]]=results[1]
    if(i%1000==0):
        print(i)
    i+= 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000


In [ ]:
individual_footprint="%s.individual_footprint" %(raw_data)

#exporting individual footprint   <customer_id;year;week;profile_id;turn;size;t1... tn >
fw=open(individual_footprint,'w')
fw.write('customer_id,year,week,profile_id,turn,size,d0t0,d1t0,d2t0,d3t0,d4t0,d5t0,d6t0,d0t1,d1t1,d2t1,d3t1,d4t1,d5t1,d6t1,d0t2,d1t2,d2t2,d3t2,d4t2,d5t2,d6t2,d0t3,d1t3,d2t3,d3t3,d4t3,d5t3,d6t3\n')
footprints=0
for uid in profiles:
    profile_id=0
    for year in profiles[uid]:
        for week in profiles[uid][year]:
            temp=np.zeros(28)
            for turn in profiles[uid][year][week]:
                d=profiles[uid][year][week][turn]
                if(turn == 0):
                    for i in range(0,7):
                        temp[i] += d[i]
                if(turn == 1):
                    for i in range(7,14):
                        temp[i] += d[i-7]
                if(turn == 2):
                    for i in range(14,21):
                        temp[i] += d[i-14]
                if(turn == 3):
                    for i in range(21,28):
                        temp[i] += d[i-21]
                
                
            fw.write(''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(turn)+','+str(sum(temp))+','
                     +str(temp[0])+','+str(temp[1])+','+str(temp[2])+','+str(temp[3])+','+str(temp[4])+','+str(temp[5])+','+str(temp[6])+','
                     +str(temp[7])+','+str(temp[8])+','+str(temp[9])+','+str(temp[10])+','+str(temp[11])+','+str(temp[12])+','+str(temp[13])+','
                     +str(temp[14])+','+str(temp[15])+','+str(temp[16])+','+str(temp[17])+','+str(temp[18])+','+str(temp[19])+','+str(temp[20])+','
                     +str(temp[21])+','+str(temp[22])+','+str(temp[23])+','+str(temp[24])+','+str(temp[25])+','+str(temp[26])+','+str(temp[27])
                     +'\n')

            profile_id = profile_id + 1
    footprints+=profile_id
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))